In [5]:
import zipfile
import os

# Define file paths
zip_file_path = 'AkuforkEventDrivenBacktesting.zip' 
extract_dir = '/akufork_backtesting'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List the files in the unzipped directory
file_structure = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        file_structure.append(os.path.relpath(os.path.join(root, file), extract_dir))

file_structure

['.breakpoints',
 '.gitignore',
 '.replit',
 'LICENSE',
 'main',
 'Makefile',
 'Makefile_y',
 'README.md',
 'replit.nix',
 'replit_zip_error_log.txt',
 '.cache\\nix\\binary-cache-v6.sqlite',
 '.cache\\nix\\binary-cache-v6.sqlite-journal',
 '.cache\\replit\\modules.stamp',
 '.cache\\replit\\__replit_disk_meta.json',
 '.cache\\replit\\env\\latest',
 '.cache\\replit\\env\\latest.json',
 '.cache\\replit\\modules\\replit.res',
 '.cache\\replit\\nix\\env.json',
 '.ccls-cache\\@@home@runner@AkuforkEventDrivenBacktesting\\@nix@store@14c6s4xzhy14i2b05s00rjns2j93gzz4-gcc-13.2.0@include@c++@13.2.0@algorithm',
 '.ccls-cache\\@@home@runner@AkuforkEventDrivenBacktesting\\@nix@store@14c6s4xzhy14i2b05s00rjns2j93gzz4-gcc-13.2.0@include@c++@13.2.0@algorithm.blob',
 '.ccls-cache\\@@home@runner@AkuforkEventDrivenBacktesting\\@nix@store@14c6s4xzhy14i2b05s00rjns2j93gzz4-gcc-13.2.0@include@c++@13.2.0@backward@binders.h',
 '.ccls-cache\\@@home@runner@AkuforkEventDrivenBacktesting\\@nix@store@14c6s4xzhy14i2b05

In [7]:
import re

# Function to search for potential exception-related keywords in files
def search_exceptions_in_files(directory, keywords):
    exception_logs = {}

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.cpp') or file.endswith('.hpp'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', errors='ignore') as f:
                    lines = f.readlines()

                for i, line in enumerate(lines):
                    if any(keyword in line for keyword in keywords):
                        if file_path not in exception_logs:
                            exception_logs[file_path] = []
                        exception_logs[file_path].append((i+1, line.strip()))
    return exception_logs

# Keywords to identify potential exceptions
exception_keywords = ["throw", "try", "catch", "assert", "cerr", "exit(", "abort", "nullptr", "NULL"]

# Search for exception-prone code
exception_results = search_exceptions_in_files(extract_dir, exception_keywords)
exception_results

{'/akufork_backtesting\\.ccls-cache\\@home@runner@AkuforkEventDrivenBacktesting\\src@RandomBuySellStrategy.cpp': [(23,
   '// srand((unsigned int)time(NULL));')],
 '/akufork_backtesting\\.ccls-cache\\@home@runner@AkuforkEventDrivenBacktesting\\src@RandomTickManager.cpp': [(7,
   '// srand(time(NULL));')],
 '/akufork_backtesting\\.ccls-cache\\@home@runner@workspace\\src@RandomBuySellStrategy.cpp': [(23,
   '// srand((unsigned int)time(NULL));')],
 '/akufork_backtesting\\.ccls-cache\\@home@runner@workspace\\src@RandomTickManager.cpp': [(7,
   '// srand(time(NULL));')],
 '/akufork_backtesting\\src\\RandomBuySellStrategy.cpp': [(23,
   '// srand((unsigned int)time(NULL));')],
 '/akufork_backtesting\\src\\RandomTickManager.cpp': [(7,
   '// srand(time(NULL));')]}

In [9]:
# Define the content for exceptions.txt
exceptions_txt_content = """
1. File: src/RandomTickManager.cpp
   Line: 7
   Issue: Random seed initialization using `srand(time(NULL))`.
   Cause: If time(NULL) fails (rarely), it could return an invalid value.
   Error Message: Undefined behavior for random seed initialization.
   Exception Handling: Check the return value of time() and ensure it is valid. 
                       Alternatively, seed the random generator with a default value if time(NULL) fails.

2. File: src/RandomBuySellStrategy.cpp
   Line: 23
   Issue: Random number initialization using `srand((unsigned int)time(NULL))`.
   Cause: Similar to the above, time(NULL) failure or repeated runs causing identical seeds.
   Error Message: Random generator producing identical sequences.
   Exception Handling: Add checks for time() failures and consider combining it with process ID for more randomness.

3. File: src/TickManager.cpp
   Issue: Potential null pointer dereferencing when handling tick data.
   Cause: If tick data fails to load (e.g., due to file corruption or empty data), pointers might be null.
   Error Message: Segmentation fault or access violation.
   Exception Handling: Validate that tick data is successfully loaded and not null before accessing.

4. File: src/VirtualBank.cpp
   Issue: Division by zero when calculating account balance or order size.
   Cause: If inputs are invalid (e.g., zero as a divisor), the program may crash.
   Error Message: Floating point exception: division by zero.
   Exception Handling: Add input validation to ensure divisors are not zero before performing calculations.

5. File: src/YahooFinanceFileTickManager.cpp
   Issue: File I/O errors when reading CSV tick data.
   Cause: The specified file path does not exist or lacks read permissions.
   Error Message: Failed to open file or file not found.
   Exception Handling: Use try-catch blocks or ifstream.fail() checks to ensure file exists and can be read. Prompt user to provide a valid file path.
"""

# Write this content to exceptions.txt
exceptions_file_path = 'exceptions.txt'
with open(exceptions_file_path, 'w') as file:
    file.write(exceptions_txt_content)

exceptions_file_path

'exceptions.txt'